In [1]:
##The premise of this project is for the implementation a CNN with VGG-16 as a feature selector 
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
#Create an ImageGenerator object that is used to randomize and make certain small transformations to the image
#to build better and robust networks

from keras.preprocessing.image import ImageDataGenerator

In [3]:
image_gen = ImageDataGenerator(rotation_range=30,
                              width_shift_range=0.1,
                              height_shift_range=0.1,
                              rescale=1/255,
                              zoom_range=0.2,
                              shear_range=0.2,
                              fill_mode='nearest')

## Model:


In [4]:
from keras.applications import vgg16
from keras.models import Sequential
from keras.layers import Dense,Dropout,Flatten,Conv2D,MaxPooling2D
from keras import optimizers

In [5]:
model = vgg16.VGG16(weights='imagenet', include_top=False, 
                    input_shape=(150,150,3), pooling="max")

for layer in model.layers[:-3]:
        layer.trainable = False

for layer in model.layers:
    print(layer, layer.trainable)


transfer_model = Sequential()
for layer in model.layers:
    transfer_model.add(layer)
transfer_model.add(Dense(128, activation="relu"))  
transfer_model.add(Dropout(0.5))
transfer_model.add(Dense(10, activation="softmax")) 

adam = optimizers.Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.00001)

transfer_model.compile(loss="categorical_crossentropy",
                      optimizer=adam,
                      metrics=["accuracy"])

58892288/58889256 [==============================] - 5s 0us/step
<tensorflow.python.keras.engine.input_layer.InputLayer object at 0x0000020EFB4D7430> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x0000020EFB4EB640> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x0000020EFB5437F0> False
<tensorflow.python.keras.layers.pooling.MaxPooling2D object at 0x0000020EFB5434C0> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x0000020EFB5C96A0> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x0000020EFB5D9100> False
<tensorflow.python.keras.layers.pooling.MaxPooling2D object at 0x0000020EFB5DBFD0> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x0000020EFB5E5CA0> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x0000020EFB5EF490> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x0000020EFB5E5A60> False
<tensorflow.python.keras.layers.pooling.Max

In [ ]:
transfer_model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 150, 150, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 150, 150, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 75, 75, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 75, 75, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 75, 75, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 37, 37, 128)       0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 37, 37, 256)       2

In [ ]:
# Load files in google colab
from google.colab import files

# Install kaggle to download the dataset
!pip install -q kaggle

# Upload the kaggle api token json file
upload = files.upload()
!mkdir ~/.kaggle
!cp /content/kaggle.json ~/.kaggle/kaggle.json
# Download the dataset from kaggle using api link
!kaggle datasets download -d slothkong/10-monkey-species

# Unzip the dataset folder
!unzip 10-monkey-species

Saving kaggle.json to kaggle.json
 98% 537M/547M [00:17<00:00, 28.7MB/s]
100% 547M/547M [00:17<00:00, 32.4MB/s]
Archive:  10-monkey-species.zip
  inflating: monkey_labels.txt       
  inflating: training/training/n0/n0018.jpg  
  inflating: training/training/n0/n0019.jpg  
  inflating: training/training/n0/n0020.jpg  
  inflating: training/training/n0/n0021.jpg  
  inflating: training/training/n0/n0022.jpg  
  inflating: training/training/n0/n0023.jpg  
  inflating: training/training/n0/n0024.jpg  
  inflating: training/training/n0/n0025.jpg  
  inflating: training/training/n0/n0026.jpg  
  inflating: training/training/n0/n0027.jpg  
  inflating: training/training/n0/n0028.jpg  
  inflating: training/training/n0/n0029.jpg  
  inflating: training/training/n0/n0030.jpg  
  inflating: training/training/n0/n0031.jpg  
  inflating: training/training/n0/n0032.jpg  
  inflating: training/training/n0/n0033.jpg  
  inflating: training/training/n0/n0034.jpg  
  inflating: training/training/n0/n0

In [8]:
train_directory = 'datasets/training/training'
validation_directory = 'datasets/validation/validation'

In [9]:
## Getting the training and the validation sets
batch_size = 16
train_gen = image_gen.flow_from_directory(train_directory,target_size=(150,150),batch_size=batch_size,
                                         class_mode='categorical')

Found 1098 images belonging to 10 classes.


In [10]:
validation_gen = image_gen.flow_from_directory(validation_directory,target_size=(150,150),batch_size=batch_size,
                                         class_mode='categorical')

Found 272 images belonging to 10 classes.


In [11]:
results = transfer_model.fit_generator(train_gen,epochs=30,steps_per_epoch=1097//batch_size,
                              validation_data=validation_gen,validation_steps=272//batch_size)

c:\users\pruth\onedrive\desktop\environment\tf_proj\lib\site-packages\tensorflow\python\keras\engine\training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/30
68/68 [==============================] - 73s 1s/step - loss: 2.8285 - accuracy: 0.1030 - val_loss: 1.9924 - val_accuracy: 0.3971
Epoch 2/30
68/68 [==============================] - 68s 997ms/step - loss: 1.9794 - accuracy: 0.3076 - val_loss: 1.6305 - val_accuracy: 0.5221
Epoch 3/30
68/68 [==============================] - 68s 1s/step - loss: 1.7016 - accuracy: 0.4126 - val_loss: 1.3917 - val_accuracy: 0.6324
Epoch 4/30
68/68 [==============================] - 69s 1s/step - loss: 1.3429 - accuracy: 0.5511 - val_loss: 1.1919 - val_accuracy: 0.7132
Epoch 5/30
68/68 [==============================] - 68s 996ms/step - loss: 1.1964 - accuracy: 0.6187 - val_loss: 1.0398 - val_accuracy: 0.7022
Epoch 6/30
68/68 [==============================] - 68s 992ms/step - loss: 1.0601 - accuracy: 0.6570 - val_loss: 0.9244 - val_accuracy: 0.7206
Epoch 7/30
68/68 [==============================] - 68s 1s/step - loss: 0.9338 - accuracy: 0.6931 - val_loss: 0.8715 - val_accuracy: 0.7537
Epoch 8/30


In [14]:
transfer_model.save('tlmonkeyCNN.h5')

In [12]:
_, acc = transfer_model.evaluate_generator(validation_gen, steps=272 //batch_size)

c:\users\pruth\onedrive\desktop\environment\tf_proj\lib\site-packages\tensorflow\python\keras\engine\training.py:1877: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  warnings.warn('`Model.evaluate_generator` is deprecated and '


In [13]:
print('The testing accuracy for the CNN with the 10-Species-Monkey dataset is : %.3f' % (acc * 100.0))

The testing accuracy for the CNN with the 10-Species-Monkey dataset is : 85.662


In [ ]:
from tensorflow import keras
x = keras.models.load_model('tlmonkeyCNN.h5')